In [1]:
# Load libraries
import pandas as pd
import numpy as np
import os
import glob
import csv

### Read files with temperature values

In [2]:
# Create a list with column names for temperature files
cols_temp = ['ob_end_time', 'max_air_temp', 'min_air_temp']

In [3]:
# Read files with minimum and maximum temperature
all_files_temp = [i for i in glob.glob('*.csv')]

#combine all files
concat_csv = pd.concat([pd.read_csv(f, skiprows=90, usecols = cols_temp)[:-1] for f in all_files_temp])
concat_csv.rename(columns = {'ob_end_time': 'date'},  inplace = True)
concat_csv['date'] = pd.to_datetime(concat_csv['date'])
concat_csv.head()

,date,max_air_temp,min_air_temp
0,1982-01-01 09:00:00,9.2,0.2
1,1982-01-02 09:00:00,12.0,0.8
2,1982-01-03 09:00:00,12.0,8.0
3,1982-01-04 09:00:00,12.2,7.5
4,1982-01-05 09:00:00,12.0,7.2


In [4]:
# Group by date
concat_index = concat_csv.set_index("date")
concat_def = concat_index.groupby(concat_index.index.date).mean().reset_index()
concat_def.rename(columns = {'index': 'date'}, inplace = True)
concat_def['date'] = pd.to_datetime(concat_def['date'])
concat_def.head()

,date,max_air_temp,min_air_temp
0,1982-01-01,9.2,0.2
1,1982-01-02,12.0,0.8
2,1982-01-03,12.0,8.0
3,1982-01-04,12.2,7.5
4,1982-01-05,12.0,7.2


In [23]:
# Check if all dates are available
all_dates = pd.DataFrame(pd.date_range(concat_def['date'].min(), concat_def['date'].max()), columns=['date'])
all_dates.head()

,date
0,1982-01-01
1,1982-01-02
2,1982-01-03
3,1982-01-04
4,1982-01-05


In [24]:
# Join all and create new columns
new_df = all_dates.merge(right=concat_def, how='left', on='date')
new_df['year'] = new_df['date'].dt.year
new_df['month'] = new_df['date'].dt.month
new_df['day'] = new_df['date'].dt.day
new_df['day_ofthe_year'] = new_df['date'].apply(lambda x:x.dayofyear)
new_df.head()

,date,max_air_temp,min_air_temp,year,month,day,day_ofthe_year
0,1982-01-01,9.2,0.2,1982,1,1,1
1,1982-01-02,12.0,0.8,1982,1,2,2
2,1982-01-03,12.0,8.0,1982,1,3,3
3,1982-01-04,12.2,7.5,1982,1,4,4
4,1982-01-05,12.0,7.2,1982,1,5,5


In [25]:
# Reorder columns
cols = ['day', 'month', 'year', 'day_ofthe_year', 'max_air_temp', 'min_air_temp']
concat_temp = new_df[cols]
concat_temp.head()

,day,month,year,day_ofthe_year,max_air_temp,min_air_temp
0,1,1,1982,1,9.2,0.2
1,2,1,1982,2,12.0,0.8
2,3,1,1982,3,12.0,8.0
3,4,1,1982,4,12.2,7.5
4,5,1,1982,5,12.0,7.2


###  Read precipitation files

In [26]:
# Read files
path = 'C:/Users/neliq/Documents/NIF/Rothamsted/projectDAYCENT_NWP/weather-METOFFICE/rain/daily'
path2 = 'C:/Users/neliq/Documents/NIF/Rothamsted/projectDAYCENT_NWP/weather-METOFFICE/rain/daily/hourly'

all_files_daily = [i for i in glob.glob(path + '/*.csv')]
all_files_hourly = [i for i in glob.glob(path2 + '/*.csv')]


cols1 = ['ob_end_time', 'prcp_amt']
cols_rain = ['ob_date', 'prcp_amt']


#combine all files
concat_rain = pd.concat([pd.read_csv(f, skiprows=61, usecols = cols_rain)[:-1] for f in all_files_daily])
concat_rain.rename(columns = {'ob_date': 'date', 'prcp_amt': 'prec_mm'},  inplace = True)
concat_rain['date'] = pd.to_datetime(concat_rain['date'])

#combine all files
concat_csv_rain2 = pd.concat([pd.read_csv(f, skiprows=61, usecols = cols1)[:-1] for f in all_files_hourly])
concat_csv_rain2.rename(columns = {'ob_end_time': 'date', 'prcp_amt': 'prec_mm'},  inplace = True)
concat_csv_rain2['date'] = pd.to_datetime(concat_csv_rain2['date'])
all_data = pd.concat([concat_rain, concat_csv_rain2])
all_data.head()

,date,prec_mm
0,1982-01-01,4.8
1,1982-01-02,0.1
2,1982-01-03,0.5
3,1982-01-04,7.8
4,1982-01-05,11.3


In [10]:
# Group by date
all_data_index = all_data.set_index("date")
all_data_grouped = all_data_index.groupby(all_data_index.index.date).sum().reset_index()
all_data_grouped.rename(columns = {'index': 'date'}, inplace = True)
all_data_grouped['date'] = pd.to_datetime(all_data_grouped['date'])
all_data_grouped['prec_cm'] = all_data_grouped['prec_mm'] * 0.1
all_data_grouped.drop(['prec_mm'], axis = 1, inplace = True)
all_data_grouped.head()

,date,prec_cm
0,1982-01-01,0.48
1,1982-01-02,0.01
2,1982-01-03,0.05
3,1982-01-04,0.78
4,1982-01-05,1.13


In [27]:
# Check if all dates are available
all_dates = pd.DataFrame(pd.date_range(all_data_grouped['date'].min(), all_data_grouped['date'].max()), columns=['date'])
all_dates.head()

,date
0,1982-01-01
1,1982-01-02
2,1982-01-03
3,1982-01-04
4,1982-01-05


In [29]:
# Join all and create new columns
all_data_dates = all_dates.merge(right=all_data_grouped, how='left', on='date')
all_data_dates['date'] = pd.to_datetime(all_data_dates['date'])
all_data_dates['year'] = all_data_dates['date'].dt.year
all_data_dates['month'] = all_data_dates['date'].dt.month
all_data_dates['day'] = all_data_dates['date'].dt.day
all_data_dates['day_ofthe_year'] = all_data_dates['date'].apply(lambda x:x.dayofyear)
all_data_dates.head()

,date,prec_cm,year,month,day,day_ofthe_year
0,1982-01-01,0.48,1982,1,1,1
1,1982-01-02,0.01,1982,1,2,2
2,1982-01-03,0.05,1982,1,3,3
3,1982-01-04,0.78,1982,1,4,4
4,1982-01-05,1.13,1982,1,5,5


In [30]:
# Reorder columns
cols = ['day', 'month', 'year', 'day_ofthe_year', 'prec_cm']
concat_rain = all_data_dates[cols]
concat_rain.head()

,day,month,year,day_ofthe_year,prec_cm
0,1,1,1982,1,0.48
1,2,1,1982,2,0.01
2,3,1,1982,3,0.05
3,4,1,1982,4,0.78
4,5,1,1982,5,1.13


In [31]:
# Merge all weather data
all_weather = concat_temp.merge(concat_rain)
all_weather['max_air_temp'] = all_weather['max_air_temp'].astype(float).round(2)
all_weather['min_air_temp'] = all_weather['min_air_temp'].astype(float).round(2)
all_weather['prec_cm'] = all_weather['prec_cm'].astype(float).round(2)
all_weather = all_weather.fillna('-99.99')
all_weather.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14610 entries, 0 to 14609
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   day             14610 non-null  int64 
 1   month           14610 non-null  int64 
 2   year            14610 non-null  int64 
 3   day_ofthe_year  14610 non-null  int64 
 4   max_air_temp    14610 non-null  object
 5   min_air_temp    14610 non-null  object
 6   prec_cm         14610 non-null  object
dtypes: int64(4), object(3)
memory usage: 913.1+ KB


In [22]:
# Export data
all_weather.to_csv("nwfp.wth", index = False, sep = '\t', header = False)

In [ ]:
# End